#### 모듈 불러오기

In [1]:
import os
import sys
import pandas as pd
from konlpy.tag import Okt
import re
from transformers import BertTokenizer, BertModel

sys.path.append(os.path.abspath('../'))
from DB import conn

c:\ProgramData\anaconda3\envs\final\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### DB에서 뉴스기사 가져오기

In [2]:
conn.connect_to_database()

MariaDB 서버에 성공적으로 연결되었습니다. 서버 버전: 10.4.34-MariaDB
현재 사용 중인 데이터베이스: cryptoStockTrading


In [3]:
select_query = '''
    select *
    from TB_KRNEWS tk 
    order by KRNEWS_DATE DESC 
    limit 20
    '''

In [4]:
conn.global_cursor.execute(select_query)

In [5]:
rows = conn.global_cursor.fetchall()

In [6]:
conn.global_cursor.description

(('KRNEWS_CODE',
  253,
  50,
  200,
  0,
  0,
  False,
  20483,
  'tk',
  'KRNEWS_CODE',
  'TB_KRNEWS'),
 ('GSTC_CODE',
  253,
  20,
  80,
  0,
  0,
  False,
  20489,
  'tk',
  'GSTC_CODE',
  'TB_KRNEWS'),
 ('INVEST_CODE',
  253,
  2,
  8,
  0,
  0,
  False,
  20489,
  'tk',
  'INVEST_CODE',
  'TB_KRNEWS'),
 ('KRNEWS_TITLE',
  253,
  100,
  400,
  0,
  0,
  False,
  4097,
  'tk',
  'KRNEWS_TITLE',
  'TB_KRNEWS'),
 ('KRNEWS_CONTENT',
  252,
  16777215,
  67108860,
  0,
  0,
  True,
  16,
  'tk',
  'KRNEWS_CONTENT',
  'TB_KRNEWS'),
 ('KRNEWS_DATE',
  12,
  4,
  19,
  0,
  0,
  True,
  16512,
  'tk',
  'KRNEWS_DATE',
  'TB_KRNEWS'),
 ('KRNEWS_PRESS',
  253,
  100,
  400,
  0,
  0,
  True,
  0,
  'tk',
  'KRNEWS_PRESS',
  'TB_KRNEWS'),
 ('KRNEWS_URL',
  253,
  1000,
  4000,
  0,
  0,
  True,
  0,
  'tk',
  'KRNEWS_URL',
  'TB_KRNEWS'))

In [7]:
columns = [i[0] for i in conn.global_cursor.description]

In [8]:
news_df = pd.DataFrame(rows, columns=columns)

In [9]:
conn.close_database_connection()

MariaDB 연결이 종료되었습니다.


### 뉴스기사 전처리

In [10]:
news_df

,KRNEWS_CODE,GSTC_CODE,INVEST_CODE,KRNEWS_TITLE,KRNEWS_CONTENT,KRNEWS_DATE,KRNEWS_PRESS,KRNEWS_URL
0,KRN20240801180008552,KR7004370003,01,[기고] 다시 대한민국에 농심(農心)을 고(告)하다,정찬호 농협중앙회 부산본부장\n\n\n\n장마가 끝나고 본격적인 무더위에 접어들면서...,2024-08-01 18:00:08,부산일보,https://n.news.naver.com/mnews/article/082/000...
1,KRN20240801174517151,KR7004370003,01,빵부터 식물성 음료까지…'수출 효자'로 떠오른 쌀 가공식품,"정부, 밀 대체 위해 가루쌀 장려신세계푸드, 우유대체 음료 출시농심은 가루쌀 건면 ...",2024-08-01 17:45:17,한국경제,https://n.news.naver.com/mnews/article/015/000...
2,KRN20240801173211902,KR7004370003,01,"계룡시, 청소년 대상 농업교실 호응",제철 농산물 활용 먹거리 체험 학습 과정 운영\n\n\n\n[계룡=뉴시스]곽상훈 기...,2024-08-01 17:32:11,뉴시스,https://n.news.naver.com/mnews/article/003/001...
3,KRN20240801165812358,KR7004370003,01,"라면업계·식약처 만났다…업계 ""규제 완화 힘써달라""",K-라면 수출 활성화를 위한 민·관 소통…농심·삼양 등 5개 업체 참석국내 주요 라...,2024-08-01 16:58:12,지디넷코리아,https://n.news.naver.com/mnews/article/092/000...
4,KRN20240801145208155,KR7004370003,01,"[단독]農心 강타한 티몬·위메프 사태…""농민·농협 피해 최소 126억원""",농림축산식품부와 농협 국회 보고 자료 입수농협 한 법인은 쌀 1600t 판매대금 못...,2024-08-01 14:52:08,아시아경제,https://n.news.naver.com/mnews/article/277/000...
5,KRN20240801141216370,KR7004370003,01,‘한강라면’ 대신 ‘센 강라면’… 올림픽 현장에 열린 ‘농심 팝업스토어’,“쎄시봉(C’est si bon) 신라면!”올림픽이 열리고 있는 프랑스 파리에서 농...,2024-08-01 14:12:16,동아일보,https://n.news.naver.com/mnews/article/020/000...
6,KRN20240801130210619,KR7004370003,01,"“하늘에서 먹는 컵라면 별미였는데”…대한항공, 일반석 제공 중단","""난기류 화상 사고 방지 위한 차원""대한항공은 오는 15일부터 장거리 노선의 기내 ...",2024-08-01 13:02:10,세계일보,https://n.news.naver.com/mnews/article/022/000...
7,KRN20240801030608954,KR7004370003,01,"“신라면 맵지만 맛있어요”… 농심, 프랑스 카르푸 팝업 성황",농심이 지난달 29일 프랑스 파리 현지 카르푸 매장에 신라면을 테마로 문을 연 팝업...,2024-08-01 03:06:08,동아일보,https://n.news.naver.com/mnews/article/020/000...
8,KRN20240731232409110,KR7042700005,01,“거품론 커지는데 3배 베팅하는 야수의 심장”...서학개미 순매수1위 보니 ‘깜놀’,"인공지능(AI) 거품론으로 미국 반도체주가 하락을 거듭하는 상황에서, 미국 주식에 ...",2024-07-31 23:24:09,매일경제,https://n.news.naver.com/mnews/article/009/000...
9,KRN20240731213101683,KR7005930003,01,"삼성전자 노사, '끝장 교섭' 결렬",사흘간 진행…이견 못 좁혀\n\n\n\n지난 29일부터 사흘간 진행된 삼성전자 노사...,2024-07-31 21:31:01,한국경제TV,https://n.news.naver.com/mnews/article/215/000...


In [11]:
news_df['KRNEWS_CONTENT'][0]

'정찬호 농협중앙회 부산본부장\n\n\n\n장마가 끝나고 본격적인 무더위에 접어들면서 논에는 벼가 무르익어가고 있지만 이를 보는 농업인의 마음은 새까맣게 타들어 가고 있다. 우리민족의 주식(主食)이요 농업의 상징으로 귀하게 대접받던 쌀의 가치가 예전만 못한 까닭이다. 80년대까지만 해도 ‘아침진지 드셨습니까?’라고 안부를 묻곤 했었지만 이제는 세상이 달라졌다.지난해 국민 1인당 연간 쌀 소비량은 56.4kg로 30년 만에 절반으로 줄어들어 성인 기준 하루 평균 한 공기 반 정도인 154.6g을 소비한다고 한다. 정부에서 생산량을 조절하고는 있지만 소비감소 폭이 압도적으로 높아 소비 활성화를 위한 특단의 대책이 시급한 가운데 쌀 소비 촉진을 위한 다양한 대책 중 ‘아침밥 먹기 운동’이 다시 주목받고 있다.질병관리청의 ‘2022년 국민건강영양조사’ 결과에 따르면 우리나라 국민의 아침식사 결식률은 34%에 달하며 주된 이유는 ‘시간이 없어서’라거나 다이어트 등 ‘건강상의 이유’가 대부분을 차지했다. 하지만 아침 식사를 거르면 두뇌 활동이 저하되고 신진대사가 원활하지 않아 오히려 체지방이 축적되고 일의 능률이 떨어질 수 있다고 한다. 득보다 실이 큰 것이다.아침 식사를 하더라도 밥이 아닌 식단인 경우도 많다. 이는 쌀에 대한 인식과도 연관이 있어 보인다. 최근 다이어트 열풍으로 탄수화물 섭취가 비만의 주요 원인이라는 인식이 확산해 쌀에 대한 부정적인 인식이 만연하다. 하지만 밥을 먹으면 살이 찐다는 것은 잘못된 편견이다. 오히려 다이어트의 첫 번째 방법이 ‘아침밥을 꼭 챙겨 먹는 것’이라 할 수 있다.쌀이 가지고 있는 탄수화물은 섬유질이 높고 나트륨, 지방 및 콜레스테롤 함량이 매우 낮기 때문에 적정량 섭취 시 다이어트 효과가 있으며, 특히 쌀눈에 많은 옥타코사놀 성분이 근육의 글리코겐양을 높여 신진대사를 활발하게 하고 스트레스를 조절하는 효과가 있다. 또한 쌀은 각종 성인병 예방에도 도움이 되는 등 알려진 것보다 훨씬 더 우수한 효능이 있어 ‘아침 식사’를 ‘밥’으

In [12]:
# 텍스트 정제 함수 정의
def clean_text(text):
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    # 소문자 변환 (영어 포함)
    text = text.lower()
    # 구두점 및 특수 문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [13]:
okt = Okt()
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

c:\ProgramData\anaconda3\envs\final\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


##### OKT

In [14]:
news_df['tokens'] = news_df['KRNEWS_CONTENT'].apply(clean_text).apply(okt.morphs)

In [15]:
news_df

,KRNEWS_CODE,GSTC_CODE,INVEST_CODE,KRNEWS_TITLE,KRNEWS_CONTENT,KRNEWS_DATE,KRNEWS_PRESS,KRNEWS_URL,tokens
0,KRN20240801180008552,KR7004370003,01,[기고] 다시 대한민국에 농심(農心)을 고(告)하다,정찬호 농협중앙회 부산본부장\n\n\n\n장마가 끝나고 본격적인 무더위에 접어들면서...,2024-08-01 18:00:08,부산일보,https://n.news.naver.com/mnews/article/082/000...,"[정찬, 호, 농협, 중앙회, 부산, 본부장, 장마, 가, 끝나고, 본격, 적, 인..."
1,KRN20240801174517151,KR7004370003,01,빵부터 식물성 음료까지…'수출 효자'로 떠오른 쌀 가공식품,"정부, 밀 대체 위해 가루쌀 장려신세계푸드, 우유대체 음료 출시농심은 가루쌀 건면 ...",2024-08-01 17:45:17,한국경제,https://n.news.naver.com/mnews/article/015/000...,"[정부, 밀, 대체, 위해, 가루, 쌀, 장려, 신세계, 푸드, 우유, 대체, 음료..."
2,KRN20240801173211902,KR7004370003,01,"계룡시, 청소년 대상 농업교실 호응",제철 농산물 활용 먹거리 체험 학습 과정 운영\n\n\n\n[계룡=뉴시스]곽상훈 기...,2024-08-01 17:32:11,뉴시스,https://n.news.naver.com/mnews/article/003/001...,"[제철, 농산물, 활용, 먹거리, 체험, 학습, 과정, 운영, 계룡, 뉴시스, 곽상..."
3,KRN20240801165812358,KR7004370003,01,"라면업계·식약처 만났다…업계 ""규제 완화 힘써달라""",K-라면 수출 활성화를 위한 민·관 소통…농심·삼양 등 5개 업체 참석국내 주요 라...,2024-08-01 16:58:12,지디넷코리아,https://n.news.naver.com/mnews/article/092/000...,"[k, 라면, 수출, 활성화, 를, 위, 한, 민관, 소통, 농심, 삼양, 등, 5..."
4,KRN20240801145208155,KR7004370003,01,"[단독]農心 강타한 티몬·위메프 사태…""농민·농협 피해 최소 126억원""",농림축산식품부와 농협 국회 보고 자료 입수농협 한 법인은 쌀 1600t 판매대금 못...,2024-08-01 14:52:08,아시아경제,https://n.news.naver.com/mnews/article/277/000...,"[농림축산식품부, 와, 농협, 국회, 보고, 자료, 입수, 농협, 한, 법인, 은,..."
5,KRN20240801141216370,KR7004370003,01,‘한강라면’ 대신 ‘센 강라면’… 올림픽 현장에 열린 ‘농심 팝업스토어’,“쎄시봉(C’est si bon) 신라면!”올림픽이 열리고 있는 프랑스 파리에서 농...,2024-08-01 14:12:16,동아일보,https://n.news.naver.com/mnews/article/020/000...,"[쎄시, 봉, cest, si, bon, 신라면, 올림픽, 이, 열리고, 있는, 프..."
6,KRN20240801130210619,KR7004370003,01,"“하늘에서 먹는 컵라면 별미였는데”…대한항공, 일반석 제공 중단","""난기류 화상 사고 방지 위한 차원""대한항공은 오는 15일부터 장거리 노선의 기내 ...",2024-08-01 13:02:10,세계일보,https://n.news.naver.com/mnews/article/022/000...,"[난, 기류, 화상, 사고, 방지, 위, 한, 차원, 대한항공, 은, 오는, 15일..."
7,KRN20240801030608954,KR7004370003,01,"“신라면 맵지만 맛있어요”… 농심, 프랑스 카르푸 팝업 성황",농심이 지난달 29일 프랑스 파리 현지 카르푸 매장에 신라면을 테마로 문을 연 팝업...,2024-08-01 03:06:08,동아일보,https://n.news.naver.com/mnews/article/020/000...,"[농심, 이, 지난달, 29일, 프랑스, 파리, 현지, 카르푸, 매장, 에, 신라면..."
8,KRN20240731232409110,KR7042700005,01,“거품론 커지는데 3배 베팅하는 야수의 심장”...서학개미 순매수1위 보니 ‘깜놀’,"인공지능(AI) 거품론으로 미국 반도체주가 하락을 거듭하는 상황에서, 미국 주식에 ...",2024-07-31 23:24:09,매일경제,https://n.news.naver.com/mnews/article/009/000...,"[인공, 지능, ai, 거품, 론, 으로, 미국, 반도체, 주가, 하락, 을, 거듭..."
9,KRN20240731213101683,KR7005930003,01,"삼성전자 노사, '끝장 교섭' 결렬",사흘간 진행…이견 못 좁혀\n\n\n\n지난 29일부터 사흘간 진행된 삼성전자 노사...,2024-07-31 21:31:01,한국경제TV,https://n.news.naver.com/mnews/article/215/000...,"[사흘, 간, 진행, 이견, 못, 좁혀, 지난, 29일, 부터, 사흘, 간, 진행,..."


In [16]:
news_df['tokens'][0]

['정찬',
 '호',
 '농협',
 '중앙회',
 '부산',
 '본부장',
 '장마',
 '가',
 '끝나고',
 '본격',
 '적',
 '인',
 '무더위',
 '에',
 '접어',
 '들면서',
 '논',
 '에는',
 '벼',
 '가',
 '무르',
 '익어가고',
 '있지만',
 '이를',
 '보는',
 '농업인',
 '의',
 '마음',
 '은',
 '새까맣게',
 '타',
 '들어',
 '가고',
 '있다',
 '우리',
 '민족',
 '의',
 '주식',
 '主食',
 '이요',
 '농업',
 '의',
 '상징',
 '으로',
 '귀하게',
 '대접받던',
 '쌀',
 '의',
 '가치',
 '가',
 '예전',
 '만',
 '못',
 '한',
 '까닭',
 '이다',
 '80년',
 '대까지만',
 '해도',
 '아침',
 '진지',
 '드셨습니까',
 '라고',
 '안부',
 '를',
 '묻곤',
 '했었지만',
 '이제',
 '는',
 '세상',
 '이',
 '달라졌다',
 '지난해',
 '국민',
 '1',
 '인당',
 '연간',
 '쌀',
 '소',
 '비량',
 '은',
 '564',
 'kg',
 '로',
 '30년',
 '만에',
 '절반',
 '으로',
 '줄어들어',
 '성인',
 '기준',
 '하루',
 '평균',
 '한',
 '공기',
 '반',
 '정도',
 '인',
 '1546',
 'g',
 '을',
 '소비',
 '한다고',
 '한다',
 '정부',
 '에서',
 '생산량',
 '을',
 '조절',
 '하고는',
 '있지만',
 '소비',
 '감소',
 '폭',
 '이',
 '압도',
 '적',
 '으로',
 '높아',
 '소비',
 '활성화',
 '를',
 '위',
 '한',
 '특단',
 '의',
 '대책',
 '이',
 '시급한',
 '가운데',
 '쌀',
 '소비',
 '촉진',
 '을',
 '위',
 '한',
 '다양한',
 '대책',
 '중',
 '아침밥',
 '먹기',
 '운동',
 '이',
 '다시',

###### 형태소 분석 후, 텍스트 데이터를 벡터화해야함
###### CounterVectorizer / TfidVectorizer 등등

##### BertTokenizer

In [17]:
# 텍스트 정제 및 토큰화
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, return_tensors=None)

In [18]:
# 데이터 토큰화
tokenized_data = news_df['KRNEWS_CONTENT'].apply(clean_text).apply(tokenize_function)

In [19]:
list_ = tokenized_data.to_list()
list_

[{'input_ids': [101, 9670, 119249, 20309, 9027, 119435, 41693, 119119, 14863, 9365, 21386, 40419, 14646, 13890, 9657, 23811, 11287, 8977, 16439, 11664, 9358, 45465, 15387, 9294, 54141, 73450, 9669, 12965, 27023, 30936, 9024, 15303, 9350, 11287, 9294, 31401, 119188, 12965, 11287, 11664, 76123, 35756, 9356, 11018, 9027, 26784, 48418, 9246, 32158, 10892, 100, 9845, 93200, 8843, 11664, 11506, 9604, 12692, 36553, 52560, 10459, 9689, 21155, 2111, 8458, 9638, 48549, 9027, 26784, 10459, 9414, 119233, 11467, 8920, 17594, 9069, 119205, 118965, 23990, 9499, 10459, 8843, 104504, 9576, 16617, 19105, 9290, 11102, 8939, 118773, 11925, 10832, 86181, 18382, 19105, 9960, 12092, 9519, 119285, 18623, 12508, 9113, 119050, 119081, 25503, 118671, 59894, 9521, 73969, 9298, 118639, 9965, 69708, 9638, 87164, 9435, 83902, 9061, 17342, 32855, 12508, 33305, 14523, 8909, 36553, 122, 12030, 21928, 9568, 18784, 9499, 9448, 29455, 44321, 10892, 61846, 47407, 11261, 10244, 10954, 9248, 10530, 9666, 30134, 11467, 9692, 

In [20]:
'''

'''

'\n\n'